<a href="https://colab.research.google.com/github/K4speeer/MB-T2/blob/master/sparkProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing PySpark on the machine.

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=92bac8f1b033f2d22290a7a244f6a4ea1d20fc355d5377e6af441d5befcf3de5
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Importing Libraries

In [58]:
from pyspark.sql import SparkSession

Creating Spark session

In [59]:
spark = SparkSession.builder.appName("ProductCategories").getOrCreate()

Loading Dataset as dataframes of Products, Categories and Relations between them.

In [60]:
products_df = spark.read.csv("products.csv", header=True, inferSchema=True)
categories_df = spark.read.csv("categories.csv", header=True, inferSchema=True)
product_category_df = spark.read.csv("products_category.csv", header=True, inferSchema=True)

View uploaded (loaded dataframes)

In [61]:
print("Products:")
products_df.show()

print("Categories:")
categories_df.show()

print("Product-Category Relationships:")
product_category_df.show()


Products:
+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|       Shirt|
|         2|       Pants|
|         3|       Shoes|
|         4|         Hat|
|         5|       Watch|
|         6|     Sandals|
|         7|        Ring|
|         8|  Eyeglasses|
|         9|       Apple|
|        10|      Orange|
+----------+------------+

Categories:
+-----------+-------------+
|category_id|category_name|
+-----------+-------------+
|          1|     Clothing|
|          2|     Footwear|
|          3|  Accessories|
|          4|      Eyewear|
|          5|   Vegetables|
+-----------+-------------+

Product-Category Relationships:
+----------+-----------+
|product_id|category_id|
+----------+-----------+
|         1|          1|
|         2|          1|
|         3|          2|
|         4|          3|
|         5|          3|
|         6|          2|
|         7|          3|
|         8|          3|
|         8|          4|
|         9|       NULL|
|  

Replace Product_ID and Category_ID with Product_Name and Category_Name respectivly

In [57]:
# Adding a new column in the left side of df containing product_name
joined_df = product_category_df.join(products_df, on='product_id', how='left')
# joined_df.show()

# Adding a category_name column to the left of the last modified df
joined_df2 = joined_df.join(categories_df, on='category_id', how='left')
# joined_df2.show()

# Creating new df with selected columns ('product_name', 'category_name')
output_df = joined_df2.select('product_name', 'category_name')
# output_df.show()

# Replacing the NULL values in category_name column with 'No Category' and printing output
output_df.fillna("No Category", subset=['category_name']).show()


+------------+-------------+
|product_name|category_name|
+------------+-------------+
|       Shirt|     Clothing|
|       Pants|     Clothing|
|       Shoes|     Footwear|
|         Hat|  Accessories|
|       Watch|  Accessories|
|     Sandals|     Footwear|
|        Ring|  Accessories|
|  Eyeglasses|  Accessories|
|  Eyeglasses|      Eyewear|
|Nose'n'Brows|  No Category|
|       Hands|  No Category|
+------------+-------------+



A function to create a new dataframe with all products and categories they belong to (if exists)

In [62]:
# The following function acts the same as previuos block of code

def prods_to_cats(products_df, categories_df, relation_df):
  """
  Analyze and cretes a dataframe of pairs Product - Category depending on Relation between them

  Args:
    products_df : Products dataframe
    categories_df : Catigories dataframe
    relation_df : The relationship between categories and products

  Returns:
    DataFrame with pairs of [Product_name - Category_name] with duplicates if the product belongs to many categories
    in addition to all products that don't belong to any category

  """

  # Adding a column that places product_name beside every product_id
  joined_df = relation_df.join(products_df, on='product_id', how='left').select('category_id', "product_name")
  # Adding a column that places category_name beside every category_id
  output_df = joined_df.join(categories_df, on='category_id', how='left').select('product_name', "category_name")
  # Replacing NULLs' with 'No Category' in category_name column
  # Returning last modified DataFrame
  return output_df.fillna('No Category', 'category_name')





In [63]:
o = prods_to_cats(products_df, categories_df, product_category_df)
o.show()

+------------+-------------+
|product_name|category_name|
+------------+-------------+
|       Shirt|     Clothing|
|       Pants|     Clothing|
|       Shoes|     Footwear|
|         Hat|  Accessories|
|       Watch|  Accessories|
|     Sandals|     Footwear|
|        Ring|  Accessories|
|  Eyeglasses|  Accessories|
|  Eyeglasses|      Eyewear|
|       Apple|  No Category|
|      Orange|  No Category|
+------------+-------------+

